In [ ]:
#| default_exp enumeration_units

# enumeration_units.py

> API details.

In [ ]:
#|hide
from nbdev.showdoc import *

ModuleNotFoundError: No module named 'nbdev'

In [ ]:
#| export
from pygris.helpers import load_tiger, validate_state, fips_codes
import pandas as pd
def counties(state = None, cb = False, resolution = '500k', year = None, cache = False):
    if year is None:
        print("Using the default year of 2020")
        year = 2020
    
    if resolution not in ['500k', '5m', '20m']:
        raise ValueError("Invalid value for resolution. Valid values are '500k', '5m', and '20m'.")
    
    if cb is True:
        if year in [1990, 2000]:
            yr = str(year)[2:]
            url = f"https://www2.census.gov/geo/tiger/PREVGENZ/co/co{yr}shp/co99_d{yr}_shp.zip"
        elif year == 2010:
            url = f"https://www2.census.gov/geo/tiger/GENZ2010/gz_2010_us_050_00_{resolution}.zip"
        elif year in [2011, 2012]:
            url = f"https://www2.census.gov/geo/tiger/GENZ{year}/cb_{year}_us_county_{resolution}.zip"            
        else:
            url = f"https://www2.census.gov/geo/tiger/GENZ{year}/shp/cb_{year}_us_county_{resolution}.zip"
            
    else:
        if year == 1990:
            raise ValueError("Please specify `cb = True` to get 1990 data.")
        elif year in [2000, 2010]:
            yr = str(year)[2:]
            url = f"https://www2.census.gov/geo/tiger/TIGER2010/COUNTY/{year}/tl_2010_us_county{yr}.zip"
        else:
            url = f"https://www2.census.gov/geo/tiger/TIGER{year}/COUNTY/tl_{year}_us_county.zip"

    ctys = load_tiger(url, cache = cache)

    if state is not None:
        valid_state = validate_state(state)
        ctys = ctys.query('STATEFP in @valid_state')

    return ctys

ImportError: attempted relative import with no known parent package